### lets start with task 1

In [1]:
from pyspark.sql import SparkSession

ModuleNotFoundError: No module named 'pyspark'

In [2]:
from pyspark.sql import functions as F

# Import for typecasting columns
from pyspark.sql.types import IntegerType,BooleanType,DateType,FloatType,StringType
from pyspark.sql.types import ArrayType

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
spark = SparkSession.builder.appName("Recipe Recommender").getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
spark

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

##### As permssions were denied whilst trying to read the file from upgrad s3, i have downloaded the file and loaded it directly into my own s3 bucket. lets read it from there and proceed

In [5]:
recipe_df = spark.read.options(header=True, inferSchema=True).csv('s3://foodproj/e-2RYFIQJPQ6ML5ST9TYHE146YH/RAW_recipes_cleaned.csv')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
recipe_df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------+-------+--------------+-------------------+--------------------+--------------------+-------+--------------------+--------------------+--------------------+-------------+
|                name|    id|minutes|contributor_id|          submitted|                tags|           nutrition|n_steps|               steps|         description|         ingredients|n_ingredients|
+--------------------+------+-------+--------------+-------------------+--------------------+--------------------+-------+--------------------+--------------------+--------------------+-------------+
|arriba   baked wi...|137739|     55|         47892|2005-09-16 00:00:00|['60-minutes-or-l...|[51.5, 0.0, 13.0,...|     11|['make a choice a...|autumn is my favo...|['winter squash',...|            7|
|a bit different  ...| 31490|     30|         26278|2002-06-17 00:00:00|['30-minutes-or-l...|[173.4, 18.0, 0.0...|      9|['preheat oven to...|this recipe calls...|['prepared pizza ...|            6|


In [7]:
assert recipe_df.count() == 231637, "There is a mistake in reading the data."
assert len(recipe_df.columns) == 12, "There is a mistake in reading the data."
assert recipe_df.schema["minutes"].dataType == IntegerType(), "The data types have not been read correctly."
assert recipe_df.schema["tags"].dataType == StringType(), "The data types have not been read correctly."
assert recipe_df.schema["n_ingredients"].dataType == IntegerType(), "The data types have not been read correctly."

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### lets move on to task 2

In [8]:
# We can remove the Square brackets by doing the following code

recipe_df = (recipe_df.withColumn('nutrition',F.regexp_replace('nutrition', '\\[|\\]', '')))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
# Now we can Split the nutrition columns, lets name it as nc_split
nc_split= F.split("nutrition",",")

# Extraction of individual names from the column, lets go with nc_name
nc_name= ["calories", "total_fat_PDV", "sugar_PDV", "sodium_PDV", "protein_PDV", "saturated_fat_PDV", "carbohydrates_PDV"]

for col_index, col_name in enumerate (nc_name):
    recipe_df= (recipe_df.withColumn(col_name,nc_split.getItem(col_index).cast(FloatType())))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
recipe_df.show(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------+-------+--------------+-------------------+--------------------+--------------------+-------+--------------------+--------------------+--------------------+-------------+--------+-------------+---------+----------+-----------+-----------------+-----------------+
|                name|    id|minutes|contributor_id|          submitted|                tags|           nutrition|n_steps|               steps|         description|         ingredients|n_ingredients|calories|total_fat_PDV|sugar_PDV|sodium_PDV|protein_PDV|saturated_fat_PDV|carbohydrates_PDV|
+--------------------+------+-------+--------------+-------------------+--------------------+--------------------+-------+--------------------+--------------------+--------------------+-------------+--------+-------------+---------+----------+-----------+-----------------+-----------------+
|arriba   baked wi...|137739|     55|         47892|2005-09-16 00:00:00|['60-minutes-or-l...|51.5, 0.0, 13.0, ...|     11|['

In [11]:
# check using assert

assert recipe_df.schema["carbohydrates_PDV"].dataType == FloatType(), "Recheck your typecasting"
assert recipe_df.collect()[123432][14] == 62.0, "The columns have not been split correctly."
assert recipe_df.collect()[10000][12] == 60.400001525878906, "The columns have not been split correctly."

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Task 3 requires us to Standardize the nutrition values

In [12]:
for nutrition_col in nc_name:
    if nutrition_col != 'calories': 
        nutrition_per_100_cal_col = nutrition_col.replace('_PDV', '') + '_per_100_cal'
        
        # Perform the transformation to calculate values per 100 calories
        recipe_df = recipe_df.withColumn(
            nutrition_per_100_cal_col,
            (F.col(nutrition_col) * 100 / F.col('calories'))
        )
        
        # Fill nulls with 0 in only the current nutrition_per_100_cal_col
        recipe_df = recipe_df.fillna({nutrition_per_100_cal_col: 0})

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
recipe_df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------+-------+--------------+-------------------+--------------------+--------------------+-------+--------------------+--------------------+--------------------+-------------+--------+-------------+---------+----------+-----------+-----------------+-----------------+---------------------+------------------+------------------+-------------------+-------------------------+-------------------------+
|                name|    id|minutes|contributor_id|          submitted|                tags|           nutrition|n_steps|               steps|         description|         ingredients|n_ingredients|calories|total_fat_PDV|sugar_PDV|sodium_PDV|protein_PDV|saturated_fat_PDV|carbohydrates_PDV|total_fat_per_100_cal| sugar_per_100_cal|sodium_per_100_cal|protein_per_100_cal|saturated_fat_per_100_cal|carbohydrates_per_100_cal|
+--------------------+------+-------+--------------+-------------------+--------------------+--------------------+-------+--------------------+---------

In [14]:
# since we still have the PDV columns, lets remove those
all_columns = recipe_df.columns
pdv_columns = [col for col in all_columns if 'PDV' in col]

recipe_df = recipe_df.drop(*pdv_columns)

recipe_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- name: string (nullable = true)
 |-- id: integer (nullable = true)
 |-- minutes: integer (nullable = true)
 |-- contributor_id: integer (nullable = true)
 |-- submitted: timestamp (nullable = true)
 |-- tags: string (nullable = true)
 |-- nutrition: string (nullable = true)
 |-- n_steps: integer (nullable = true)
 |-- steps: string (nullable = true)
 |-- description: string (nullable = true)
 |-- ingredients: string (nullable = true)
 |-- n_ingredients: integer (nullable = true)
 |-- calories: float (nullable = true)
 |-- total_fat_per_100_cal: double (nullable = false)
 |-- sugar_per_100_cal: double (nullable = false)
 |-- sodium_per_100_cal: double (nullable = false)
 |-- protein_per_100_cal: double (nullable = false)
 |-- saturated_fat_per_100_cal: double (nullable = false)
 |-- carbohydrates_per_100_cal: double (nullable = false)

In [15]:
# Total fat check for id 28881
assert recipe_df.filter("id == 28881").select('total_fat_per_100_cal').first()[0] == 0, "total_fat_per_100_cal for recipe 28881 should be 0"

# Total fat check for id 112140
assert round(recipe_df.filter("id == 112140").select('total_fat_per_100_cal').first()[0]) == 8, "total_fat_per_100_cal for recipe 112140 should be 8"

# Checking for nulls
for c in ['total_fat_per_100_cal', 'sugar_per_100_cal', 'sodium_per_100_cal', 
          'protein_per_100_cal', 'saturated_fat_per_100_cal', 'carbohydrates_per_100_cal']:
    assert recipe_df.select(F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c)).collect()[0][0] == 0, "There are Nulls in the data"


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### In Task 4 we need to convert the 'Tags' column from string into an array of strings

In [26]:
print(recipe_df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[name: string, id: int, minutes: int, contributor_id: int, submitted: timestamp, tags: array<string>, nutrition: string, n_steps: int, steps: string, description: string, ingredients: string, n_ingredients: int, calories: float, total_fat_per_100_cal: double, sugar_per_100_cal: double, sodium_per_100_cal: double, protein_per_100_cal: double, saturated_fat_per_100_cal: double, carbohydrates_per_100_cal: double]

In [16]:
from pyspark.sql.functions import regexp_replace, split

recipe_df = (recipe_df
                  .withColumn('tags', 
                             regexp_replace('tags', '[\\[\\]\']', '')) 
                  .withColumn('tags', 
                             split('tags', ','))  # Split the cleaned string into an array
                 )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
recipe_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------+-------+--------------+-------------------+--------------------+--------------------+-------+--------------------+--------------------+--------------------+-------------+--------+---------------------+------------------+------------------+-------------------+-------------------------+-------------------------+
|                name|    id|minutes|contributor_id|          submitted|                tags|           nutrition|n_steps|               steps|         description|         ingredients|n_ingredients|calories|total_fat_per_100_cal| sugar_per_100_cal|sodium_per_100_cal|protein_per_100_cal|saturated_fat_per_100_cal|carbohydrates_per_100_cal|
+--------------------+------+-------+--------------+-------------------+--------------------+--------------------+-------+--------------------+--------------------+--------------------+-------------+--------+---------------------+------------------+------------------+-------------------+-------------------------+----

In [18]:
recipe_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- name: string (nullable = true)
 |-- id: integer (nullable = true)
 |-- minutes: integer (nullable = true)
 |-- contributor_id: integer (nullable = true)
 |-- submitted: timestamp (nullable = true)
 |-- tags: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- nutrition: string (nullable = true)
 |-- n_steps: integer (nullable = true)
 |-- steps: string (nullable = true)
 |-- description: string (nullable = true)
 |-- ingredients: string (nullable = true)
 |-- n_ingredients: integer (nullable = true)
 |-- calories: float (nullable = true)
 |-- total_fat_per_100_cal: double (nullable = false)
 |-- sugar_per_100_cal: double (nullable = false)
 |-- sodium_per_100_cal: double (nullable = false)
 |-- protein_per_100_cal: double (nullable = false)
 |-- saturated_fat_per_100_cal: double (nullable = false)
 |-- carbohydrates_per_100_cal: double (nullable = false)

In [19]:
# Code check cell
# Do not edit cells with assert commands
# If an error is shown after running this cell, please recheck your code.  

assert recipe_df.schema["tags"].dataType == ArrayType(StringType(), True), "You have not split the string into an array."
assert recipe_df.collect()[2][5] == ['time-to-make','course', 'preparation', 'main-dish', 'chili', 'crock-pot-slow-cooker', 'dietary', 'equipment', '4-hours-or-less'], "Recheck your string cleaning and splitting operations."


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
You have not split the string into an array.
Traceback (most recent call last):
AssertionError: You have not split the string into an array.



In [19]:
recipe_dff = spark.read.options(header=True, inferSchema=True).csv('s3://foodproj/e-2RYFIQJPQ6ML5ST9TYHE146YH/RAW_recipes_cleaned.csv')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
You have not split the string into an array.
Traceback (most recent call last):
AssertionError: You have not split the string into an array.

